In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

In [19]:
base_dir = os.path.join(os.path.expanduser('~'), 'Downloads', 'Deepdive')


Applying Histogram Equalization and denoising

In [20]:
def preprocess_image(img):
    # Convert image to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)

    # Merge the LAB channels and convert back to RGB
    lab = cv2.merge((l, a, b))
    img_clahe = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    # Apply denoising (fastNLMeans)
    denoised_img = cv2.fastNlMeansDenoisingColored(img_clahe, None, 10, 10, 7, 21)

    return denoised_img

# Custom preprocessing function for ImageDataGenerator
def custom_preprocess(img):
    # Convert PIL image to numpy array
    img_array = np.array(img)

    # Apply preprocessing
    processed_img = preprocess_image(img_array)

    return processed_img


Data Augmentation

In [21]:
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.15,  # Apply shearing transformations
    zoom_range=0.2,  # Randomly zoom into images
    horizontal_flip=True,  # Randomly flip images horizontally
    preprocessing_function=custom_preprocess  # Apply custom preprocessing
)

Loading data


In [22]:
train_data = datagen.flow_from_directory(
    os.path.join(base_dir, 'Train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    os.path.join(base_dir, 'Validation'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = datagen.flow_from_directory(
    os.path.join(base_dir, 'Test'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2667 images belonging to 33 classes.
Found 667 images belonging to 33 classes.
Found 660 images belonging to 33 classes.
